# Explore `meta-kaggle` Dataset

## --- Data Clean

Pre-requisites: 

1. Install Python package `kaggle`
2. [Create token](https://www.kaggle.com/docs/api) and 
3. Put the `kaggle.json` file to the `.kaggle` folder of your home directory
4. Download dataset `meta-kaggle` if haven't yet, by excuting: 
   ```python
   import api
   api.download_rawdata(rawdata_path)
   ```

In [1]:
# std library
import os
from datetime import datetime

# third-party
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

# local
from datafilter import table_filter

In [2]:
rawdata_path = os.path.abspath('./__rawdata__')

### 1. Mapping Datasets

Create a dataset of submissions, containing the following fields:
- Id 
- TeamId 
- CompetitionId 
- SubmissionDate
- PublicScore
- PrivateScore

In [3]:
# Submissions
# - TeamId
# - SubmissionDate
tbl_submissions = table_filter( \
	'Submissions', rawdata_path,
	fields = ['Id', 'TeamId', 'SubmissionDate', 'IsAfterDeadline',
		'PublicScoreLeaderboardDisplay', 'PrivateScoreFullPrecision'],
	fields_index = ['Id'],
	fields_datetime = ['SubmissionDate'],
)
# drop all submissions after deadlines
tbl_submissions = tbl_submissions[~tbl_submissions['IsAfterDeadline']]
tbl_submissions.drop('IsAfterDeadline', axis=1, inplace=True)
# rename long name
tbl_submissions.rename(columns={
	'PublicScoreLeaderboardDisplay': 'PublicScore',
	'PrivateScoreFullPrecision': 'PrivateScore'
}, inplace=True)

In [4]:
# Mapping: Teams
# - CompetitionId
map_teams_competitions = table_filter( \
	'Teams', rawdata_path,
	fields = ['Id', 'CompetitionId'],
	fields_index = ['Id'],
)

In [5]:
# Merge `Submissions` and `TeamsCompetition`
tbl_submissions = pd.merge(tbl_submissions, map_teams_competitions,
	how = 'left', left_on = 'TeamId', right_on = 'Id', sort = False,
	suffixes=('', '_Team'))
tbl_submissions.drop(columns='Id_Team', inplace=True)
tbl_submissions.dtypes

Id                         int64
TeamId                     int64
SubmissionDate    datetime64[ns]
PublicScore              float64
PrivateScore             float64
CompetitionId              int64
dtype: object

### 2. Splitting Contests by Types

In [6]:
# Table of Contests
tbl_contests = table_filter( \
	'Competitions', rawdata_path,
	fields=['Id',
		'EnabledDate', 'DeadlineDate', 'ProhibitNewEntrantsDeadlineDate',
		'TeamMergerDeadlineDate', 'TeamModelDeadlineDate', 'TeamModelDeadlineDate',
		'ModelSubmissionDeadlineDate',
		'HasLeaderboard', 'LeaderboardPercentage', 'MaxDailySubmissions',
		'RewardType', 'RewardQuantity', 'NumPrizes',
		'FinalLeaderboardHasBeenVerified'
	],
	fields_datetime=[
		'EnabledDate', 'DeadlineDate', 'ProhibitNewEntrantsDeadlineDate',
		'TeamMergerDeadlineDate', 'TeamModelDeadlineDate', 'TeamModelDeadlineDate',
		'ModelSubmissionDeadlineDate'
	],
	fields_index=['Id'],
)
len(tbl_contests)

9446

In [7]:
print('Filter 1: How many contests are there having records of submissions from players?')
lst_contests_with_submissions = tbl_submissions['CompetitionId'].unique()
print(len(lst_contests_with_submissions))

# Filter 1
tbl_contests = tbl_contests[tbl_contests['Id'].isin(lst_contests_with_submissions)]
tbl_submissions = tbl_submissions.loc[tbl_submissions['CompetitionId'].isin(tbl_contests['Id'])]

Filter 1: How many contests are there having records of submissions from players?
5694


In [8]:
print('Filter 2: In above, how many contests are there having public leaderboard?')
lst_contests_with_leaderboard = tbl_contests.loc[tbl_contests['HasLeaderboard']==True]['Id']
print(len(lst_contests_with_submissions))

# Filter 2
tbl_contests = tbl_contests[tbl_contests['Id'].isin(lst_contests_with_leaderboard)]
tbl_submissions = tbl_submissions.loc[tbl_submissions['CompetitionId'].isin(tbl_contests['Id'])]

Filter 2: In above, how many contests are there having public leaderboard?
5694


In [9]:
print('Is there never-ending contests?')
forever_date = datetime(2029, 1, 1, 0, 0, 0)
print((tbl_contests['DeadlineDate'] > forever_date).sum())
print((tbl_contests['DeadlineDate'].isna()).sum())

Is there never-ending contests?
0
0


In [10]:
print('How many reward types are there?')
print(tbl_contests['RewardType'].value_counts(dropna=False))

How many reward types are there?
RewardType
NaN          4778
USD           407
Knowledge     381
Swag           95
Jobs           14
Kudos          10
Prizes          6
EUR             1
Name: count, dtype: int64


In [11]:
print('List of the number of contests providing multiple prize:')
tbl_contests['NumPrizes'].value_counts()

List of the number of contests providing multiple prize:


NumPrizes
1     4722
0      498
3      292
5       82
4       28
10      18
6       16
2       11
8       10
7        9
9        4
13       1
12       1
Name: count, dtype: int64

In [12]:
# Split contests with prize type
lst_contest_Usd = tbl_contests.loc[tbl_contests['RewardType']=='USD']['Id']
lst_contest_Knowledge = tbl_contests.loc[tbl_contests['RewardType']=='Knowledge']['Id']
lst_contest_Swag = tbl_contests.loc[tbl_contests['RewardType']=='Swag']['Id']
lst_contest_Kudos = tbl_contests.loc[tbl_contests['RewardType']=='Kudos']['Id']
lst_contest_EUR = tbl_contests.loc[tbl_contests['RewardType']=='EUR']['Id']

# Split contests with prize number
lst_contest_1_prize = tbl_contests.loc[tbl_contests['NumPrizes']==1]['Id']
lst_contest_2_prize = tbl_contests.loc[tbl_contests['NumPrizes']==2]['Id']
lst_contest_3_prize = tbl_contests.loc[tbl_contests['NumPrizes']==3]['Id']
lst_contest_4_prize = tbl_contests.loc[tbl_contests['NumPrizes']==4]['Id']
lst_contest_5_prize = tbl_contests.loc[tbl_contests['NumPrizes']==5]['Id']
lst_contest_6_prize = tbl_contests.loc[tbl_contests['NumPrizes']==6]['Id']
lst_contest_7_prize = tbl_contests.loc[tbl_contests['NumPrizes']==7]['Id']
lst_contest_8_prize = tbl_contests.loc[tbl_contests['NumPrizes']==8]['Id']
lst_contest_9_prize = tbl_contests.loc[tbl_contests['NumPrizes']==9]['Id']

### 3. Select contest providing single USD prize

In [13]:
# select
tbl_contest_1_Usd_prize = tbl_contests[tbl_contests['Id'].isin(lst_contest_Usd)]
tbl_contest_1_Usd_prize = tbl_contest_1_Usd_prize[tbl_contest_1_Usd_prize['Id'].isin(lst_contest_1_prize)]

# filter
tbl_contest_1_Usd_prize = tbl_contest_1_Usd_prize.loc[tbl_contest_1_Usd_prize['RewardQuantity'] > 0]
lst_contest_1_Usd_prize = tbl_contest_1_Usd_prize['Id']
len(lst_contest_1_Usd_prize)

42

In [14]:
lst_contest_1_Usd_prize.values

array([2435, 2445, 2448, 2452, 2454, 2464, 2467, 2478, 2479, 2487, 2488,
       2496, 2549, 2589, 2762, 2860, 2895, 2963, 3065, 3294, 3364, 3370,
       3377, 3385, 3386, 3469, 3493, 3507, 3521, 3526, 3586, 3706, 3867,
       3928, 3973, 3984, 4195, 4378, 4383, 4493, 4495, 4704])

In [20]:
tbl_contest_1_Usd_prize.head()

,Id,EnabledDate,DeadlineDate,ProhibitNewEntrantsDeadlineDate,TeamMergerDeadlineDate,TeamModelDeadlineDate,ModelSubmissionDeadlineDate,FinalLeaderboardHasBeenVerified,HasLeaderboard,LeaderboardPercentage,MaxDailySubmissions,RewardType,RewardQuantity,NumPrizes
1,2435,2010-04-27 21:29:09,2010-08-02 12:32:00,NaT,NaT,NaT,NaT,True,True,30,4,USD,500.0,1
5,2445,2010-12-13 09:22:46,2011-02-20 22:00:00,NaT,NaT,NaT,NaT,True,True,25,2,USD,5000.0,1
7,2448,2010-08-09 04:19:02,2010-09-19 23:00:00,NaT,NaT,NaT,NaT,True,True,20,2,USD,500.0,1
8,2452,2010-09-20 08:38:44,2010-11-21 23:00:00,NaT,NaT,NaT,NaT,True,True,20,3,USD,500.0,1
9,2454,2010-10-10 04:00:28,2011-02-08 09:00:00,NaT,NaT,NaT,NaT,True,True,62,2,USD,150.0,1


In [21]:
tbl_submissions_selected = tbl_submissions.loc[tbl_submissions['CompetitionId'].isin(lst_contest_1_Usd_prize)]
tbl_submissions_selected.head()

,Id,TeamId,SubmissionDate,PublicScore,PrivateScore,CompetitionId
0,2180,496,2010-04-29,55.76919,56.213902,2435
1,2181,497,2010-04-30,47.11539,50.000000,2435
2,2182,497,2010-04-30,61.05770,65.606903,2435
3,2184,499,2010-05-01,47.11539,50.000000,2435
4,2185,500,2010-05-02,61.05770,62.283199,2435


#### Contest `2435` for example

In [17]:
tbl_submissions_2435 = tbl_submissions_selected.loc[tbl_submissions_selected['CompetitionId']==2435]

In [22]:
tbl_submissions_2435.head()

,Id,TeamId,SubmissionDate,PublicScore,PrivateScore,CompetitionId
0,2180,496,2010-04-29,55.76919,56.213902,2435
1,2181,497,2010-04-30,47.11539,50.000000,2435
2,2182,497,2010-04-30,61.05770,65.606903,2435
3,2184,499,2010-05-01,47.11539,50.000000,2435
4,2185,500,2010-05-02,61.05770,62.283199,2435


In [23]:
print('How many teams are there in this contest?')
print(tbl_submissions_2435['TeamId'].unique().size)

print('How many submission are there in total?')
print(len(tbl_submissions_2435))

print('List the number of submissions for the most active 5 teams:')
#tbl_submissions_2435.groupby()

How many teams are there in this contest?
107
How many submission are there in total?
855
List the number of submissions for the most active 5 teams:
